In [484]:
import pandas as pd
import numpy as np 
import imageio as iio
from PIL import  Image
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential 
from keras.optimizers import Adam
import keras
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


In [485]:
MAX_PICTURES = 1000

In [486]:
df = pd.read_csv('merged.csv', index_col=0)

In [487]:
df = df.sample(frac=1).reset_index(drop=True)

In [488]:
# read first 100 images from dataframe 
image_list = []
for image in df['image_path'][0:MAX_PICTURES]:
    im = Image.open(image)
    im = im.resize((100,75))
    a = np.asarray(im)
    image_list.append(a)


In [489]:
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000655,ISIC_0028283,bcc,histo,70.0,female,neck,data/ham10000_images_part_1\ISIC_0028283.jpg
1,HAM_0005334,ISIC_0031298,bcc,histo,75.0,male,upper extremity,data/ham10000_images_part_2\ISIC_0031298.jpg
2,HAM_0002644,ISIC_0029417,akiec,histo,80.0,female,neck,data/ham10000_images_part_2\ISIC_0029417.jpg
3,HAM_0001397,ISIC_0026796,mel,histo,70.0,male,back,data/ham10000_images_part_1\ISIC_0026796.jpg
4,HAM_0002194,ISIC_0025814,mel,histo,65.0,male,back,data/ham10000_images_part_1\ISIC_0025814.jpg
...,...,...,...,...,...,...,...,...
10010,HAM_0000952,ISIC_0024546,mel,histo,80.0,female,lower extremity,data/ham10000_images_part_1\ISIC_0024546.jpg
10011,HAM_0004753,ISIC_0031884,nv,follow_up,50.0,female,back,data/ham10000_images_part_2\ISIC_0031884.jpg
10012,HAM_0000277,ISIC_0029182,nv,histo,60.0,male,chest,data/ham10000_images_part_1\ISIC_0029182.jpg
10013,HAM_0003661,ISIC_0026910,nv,follow_up,35.0,female,trunk,data/ham10000_images_part_1\ISIC_0026910.jpg


In [490]:
mel_dict = {
    'akiec': 'Actinic keratosis-like lesions',
    'bcc': 'basal cell carcinoma',
    'bkl': 'benign keratosis',
    'df': 'dermatofibroma',
    'mel': 'melanoma',
    'nv': 'melanocytic nevi',
    'vasc': 'vascular lesions / hemorrhage'
}

In [491]:
y = df['dx'][0:MAX_PICTURES]
y

0        bcc
1        bcc
2      akiec
3        mel
4        mel
       ...  
995       nv
996      bkl
997       nv
998    akiec
999       nv
Name: dx, Length: 1000, dtype: object

In [492]:
# map diagnoses to dictionary
mel_dict = {
    'akiec': 0,
    'bcc': 1,
    'bkl': 2,
    'df': 3,
    'mel': 4,
    'nv': 5,
    'vasc': 6
}

In [493]:
image_list = np.asarray(image_list)
y = y.map(mel_dict)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(image_list, y, test_size=0.20, random_state=42)


In [494]:
# ALWAYS CONVERT AN ARRAY TO AN NUMPY ARRAY
y = np.asarray(y) 

In [495]:
y.shape

(1000, 7)

In [496]:
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(75, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(75, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [497]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [498]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 75, 100, 75)       2100      
                                                                 
 conv2d_61 (Conv2D)          (None, 73, 98, 75)        50700     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 36, 49, 75)       0         
 g2D)                                                            
                                                                 
 dropout_45 (Dropout)        (None, 36, 49, 75)        0         
                                                                 
 conv2d_62 (Conv2D)          (None, 34, 47, 64)        43264     
                                                                 
 conv2d_63 (Conv2D)          (None, 32, 45, 64)        36928     
                                                     

In [499]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

c:\Users\ozcan\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [500]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [501]:
epochs = 10
batch_size = 5


model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, callbacks=[learning_rate_reduction], validation_data=(X_test, y_test))

Epoch 1/10
160/160 [==============================] - 20s 123ms/step - loss: 7.5857 - accuracy: 0.6137 - val_loss: 1.7545 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 2/10
160/160 [==============================] - ETA: 0s - loss: 1.2999 - accuracy: 0.6562

In [ ]:
model.evaluate(x=X_test, y=y_test)

1/1 [==============================] - 0s 226ms/step - loss: 1.0227 - accuracy: 0.5500


[1.0227314233779907, 0.550000011920929]